In [ ]:
#作業ディレクトリを取得し、HOMEに格納
import os
HOME = os.getcwd()
print(HOME)

In [ ]:
#Ultralytics YOLOv8 をインストールし、インストールが成功したか確認
!pip install ultralytics == 8.0.196

from IPython import display
display.clear_output()

import ultralytics
ultralytics.checks()

In [ ]:
#必要なモジュールをインポート
from ultralytics import YOLO

from IPython.display import display, Image

In [ ]:
#roboflowから魚のデータをダウンロード
from roboflow import Roboflow
rf = Roboflow(api_key="jSLqWbf9yKIHEIR7GrtN")
project = rf.workspace("kota-h6pkq").project("fish-pgpgz")
version = project.version(1)
dataset = version.download("yolov8")

In [ ]:
#ダウンロードしたデータセットの保存先パスを確認
dataset.location

In [1]:
#データセットをYOLOv8s-segモデルでトレーニングし、resultsに結果を格納
model = YOLO('yolov8s-seg.pt')
results = model.train(data="/home/nakahira/workspace/Nishida/fish/seavis1003/fish-1/data.yaml", batch=4, epochs=300, imgsz=640)

NameError: name 'YOLO' is not defined

In [ ]:
#トレーニング結果の画像を表示
Image(filename=f'/home/nakahira/workspace/Nishida/fish/seavis1003/runs/segment/train/results.png', width=600)

In [ ]:
#fish count(Video)

#必要なライブラリをインポート
import cv2
import numpy as np

#YOLOモデルのファイルのパスを指定し、そのモデルを読み込む
model_file = '/home/nakahira/workspace/Nishida/fish/seavis1003/runs/segment/train/weights/best.pt'
model = YOLO(model_file)

#モデルのクラス名を取得
class_names = model.names

#クラスごとの色を指定
colors = [(255,0,0), (0,255,0),(0,0,255)]

#処理するビデオファイルのパスを指定し、そのビデオを読み込む
video_path = '/home/nakahira/workspace/nakahira/seavis/OnnaDataset/GH012962.MP4'
cap = cv2.VideoCapture(video_path)

#出力ビデオの設定を行い、保存先とフォーマットを指定
fourcc = cv2.VideoWriter_fourcc(*'mp4v')
out = cv2.VideoWriter('/home/nakahira/workspace/Nishida/fish/seavis1003/runs/segment/predict/output_video8.mp4', fourcc, 30.0, (int(cap.get(3)),int(cap.get(4))))

#フレームごとの魚のカウントを保持するリストを初期化
frame_fish_counts = []

#ビデオが開いている間、フレームを読み込む
while cap.isOpened():
    ret, frame = cap.read()
    if not ret : #フレームが読み込めない場合はループを終了
        break

    #フレームに対してYOLOモデルを適用し、結果を取得
    results = model(frame)

    #魚のカウントとクラスごとのカウントを初期化
    fish_count = 0
    class_counts = {}

    #各検出結果に対してループを開始
    for result in results:
        for detection in result.boxes:
            class_id = int(detection.cls.item())
            x1,y1,x2,y2 = map(int, detection.xyxy[0])
            confidence = detection.conf[0]
            label = class_names[class_id]

            #正誤認識が0.3以上の場合、魚のカウントを1増やし、クラス名を取得
            if confidence > 0.3:
                fish_count += 1
                class_name = class_names[int(class_id)]

                #クラス名が存在しない場合、0に初期化
                if class_name not in class_counts:
                    class_counts[class_name] = 0

                #検出された魚の周りに矩形を描画し、ラベルを表示
                class_counts[class_name] += 1
                color = colors[int(class_id) % len(colors)]
                cv2.rectangle(frame, (int(x1), int(y1)),(int(x2), int(y2)), color, 2)
                label = f'{class_names[int(class_id)]}:{confidence:.2f}'
                cv2.putText(frame, label,(int(x1),int(y1) - 10),cv2.FONT_HERSHEY_SIMPLEX, 0.5, color, 2)

    #各クラスのカウントをフレームに表示
    y_offset = 30
    for class_name, count in class_counts.items():
        cv2.putText(frame, f'{class_name}:{count}', (10, y_offset), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2)
        y_offset += 30

    #フレームごとの魚のカウントをリストに追加し、出力ビデオにフレームを書き込む
    frame_fish_counts.append(class_counts)
    out.write(frame)

cap.release()
out.release()
cv2.destroyAllWindows()

#フレームごとの魚のカウントをファイルに保存
np.save('frame_fish_counts.npy', frame_fish_counts)

In [ ]:
#必要なライブラリをインポート
import numpy as np
import matplotlib.pyplot as plt

#保存された魚のカウントデータをファイルから読み込む
frame_counts = np.load('frame_fish_counts.npy', allow_pickle = True)

#図のサイズを設定
plt.figure(figsize = (10, 5))

#クラスごとの色を指定
colors = ['b','g','r','c','m','y','k']

#すべてのフレームのクラス名を集めて1つのセットにする
all_class_names = set()
for frame in frame_counts:
    all_class_names.update(frame.keys())

#クラスの数に応じてサブプロットを作成
num_classes = len(all_class_names)
fig, axes = plt.subplots(num_classes, 1, figsize = (10, 5 * num_classes))

#各クラスに対してループを開始し、フレームごとの魚のカウントをプロット
for i, (class_name, color) in enumerate (zip(all_class_names, colors)):
    counts = [frame.get(class_name, 0) for frame in frame_counts]
    axes[i].plot(counts, label = class_name, color = color)

    #x軸とy軸のラベルを設定
    axes[i].set_xlabel('Frame_Number')
    axes[i].set_ylabel('Fish_Count')

    #グラフのタイトルを設定
    axes[i].set_title(f'Fish Count per Frame:{class_name}')
    axes[i].legend()

plt.tight_layout()
plt.grid(True)

#グラフを指定したパスに保存
plt.savefig('/home/nakahira/workspace/Nishida/fish/seavis1003/runs/segment/plot_graph/output_graph_Fish.png')